
# Shared rides with pricing - Delft 

 ### Choice Function (Deterministic):
 `pool_price.py`
 * Pickup Distance: distance from driver initial position to the first pickup point
 * Travel Distance: distance from driver's initial position to the drop off point of the last passenger
 * Operating Cost: This include all the expenses
 * Profit: Driver revenue to serve the request
            
 
  ### KPI:
   
   * Profit of Individual driver
   * Profit of all the drivers
   * No.of rejected rides
   * U - PAX (Utility) 
  
   ### TBD- Choice Function (Probablistic):
  
  * choice logic to be applied inside `pool_price.py` 
  * P(R)= exp(beta * Profit_R)/ sum_all the rides( exp(beta * Profit_R)
 



-------------------------------------------------------------------------------------------------------

# Pricing and Driver Earnings for a Two-Sided Mobility Platform: A Case of Amsterdam, the Netherlands

or 

# The Effects of Profit-Based Pricing on Driver Earnings and Performance of Two-Sided Mobility Platforms

# Abstract  

In this paper, we investigate how the  pricing of ride-pooling affects driver earnings. We also examine how profit-based setting affects these performance indicators. To this end, we applied a matching algorithm  to the case of ride-pooling and give a choice set to the driver for the case of Amsterdam, the Netherlands. For our simulation, we utilize an agent-based simulator reproducing the transport systems for two-sided mobility platforms (like Uber and Lyft) and applied three state-of-the-art pricing strategies such as <strong>profit maximization</strong>,  <strong>solo ride-hailing</strong>, and <strong>nearest pickup ride-pooling</strong>. We find that the profit maximization pricing strategy outperforms the other and traveler utility can be further improved by $\%X$ while reducing the total cost to serve the pooled rides. While offering a discount for profit maximization travel time is significantly higher $\%X$  than for private rides. 

-------------------------------------------------------------------------------------------------------

## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo ride-hailing</strong>

<strong>3. Nearest pickup ride-pooling</strong>

## Load ExMAS and MaaSSim


In [98]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)    
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [99]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 30 # vehicles



## Parameters for ExMAS

In [100]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



In [101]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

30-05-23 11:51:22-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
30-05-23 11:51:22-INFO-creating combinations
30-05-23 11:51:22-INFO-9900	 nR*(nR-1)
30-05-23 11:51:22-INFO-Reduction of feasible pairs by 99.71%
30-05-23 11:51:22-INFO-Degree 2 	Completed
30-05-23 11:51:22-INFO-trips to extend at degree 2 : 125
30-05-23 11:51:22-INFO-At degree 2 feasible extensions found out of 4 searched
30-05-23 11:51:22-INFO-Degree 3 	Completed
30-05-23 11:51:22-INFO-trips to extend at degree 3 : 4
30-05-23 11:51:22-INFO-At degree 3 feasible extensions found out of 0 searched
30-05-23 11:51:22-INFO-Degree 4 	Completed
30-05-23 11:51:22-INFO-Max degree reached 4
30-05-23 11:51:22-INFO-Trips still possible to extend at degree 4 : 0


In [102]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

30-05-23 11:51:22-INFO-Matching 100 trips to 229 rides in order to min u_veh
30-05-23 11:51:22-INFO-Problem solution: Optimal. 
Total costs for single trips:         26,382 
reduced by matching to:               23,321


In [103]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],1.675500,90,1,[1.6755],"[0, 90]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44759522, 44738668]","[None, 0, 0]",node time req_id od 0 NaN...,90,0.90,1.360500,0.272100,1.0884
1,[1],3.703000,200,1,[3.7030000000000003],"[57, 200]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 4239889943, 44778412]","[None, 1, 1]",node time req_id od 0 ...,200,2.00,3.003000,0.600600,2.4024
2,[2],1.319500,71,1,[1.3195],"[79, 71]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44819251, 559635307]","[None, 2, 2]",node time req_id od 0 N...,71,0.71,1.071000,0.214200,0.8568
3,[3],2.355500,127,1,[2.3555],"[138, 127]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44871803, 1436427147]","[None, 3, 3]",node time req_id od 0 ...,127,1.27,1.911000,0.382200,1.5288
4,[4],5.089000,275,1,[5.089],"[197, 275]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44836387, 44813101]","[None, 4, 4]",node time req_id od 0 NaN...,275,2.75,4.126500,0.825300,3.3012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,"[71, 69]",10.539300,483,21,"[7.693250000000001, 2.84605]","[2413.5, 84, 154, 245]","[71, 69]","[69, 71]",2,224,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1413910851, 4349049085, 1530303598, 430...","[None, 71, 69, 69, 71]",node time req_id od 0 ...,483,4.83,6.826500,1.365300,5.4612
225,"[32, 33, 37]",17.145550,728,30,"[5.615825, 6.427199999999999, 5.102525]","[1240.5, 58, 344, 26, 81, 219]","[32, 33, 37]","[32, 33, 37]",3,225,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 610983537, 629532228, 4349049055, 33772...","[None, 32, 33, 37, 32, 33, 37]",node time req_id od 0 ...,728,7.28,11.464875,2.292975,9.1719
226,"[68, 66, 71]",17.205300,827,30,"[4.837125, 4.33595, 8.032225]","[2260.0, 103, 147, 101, 55, 421]","[68, 66, 71]","[68, 66, 71]",3,226,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1566353769, 1608998249, 1413910851, 465...","[None, 68, 66, 71, 68, 66, 71]",node time req_id od 0 ...,827,8.27,10.899000,2.179800,8.7192
227,"[33, 32, 37]",16.993125,702,30,"[6.6114749999999995, 5.2996, 5.082050000000001]","[1243.0, 53, 323, 26, 81, 219]","[33, 32, 37]","[32, 33, 37]",3,227,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 629532228, 610983537, 4349049055, 33772...","[None, 33, 32, 37, 32, 33, 37]",node time req_id od 0 ...,702,7.02,11.464875,2.292975,9.1719


In [104]:
params.kpi = 1

In [105]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

30-05-23 11:51:24-INFO-Matching 100 trips to 229 rides in order to min u_veh
30-05-23 11:51:24-INFO-Problem solution: Optimal. 
Total costs for single trips:         26,382 
reduced by matching to:               23,321
0.5
0.5
0.5
0.5
0.5
30-05-23 11:51:25-CRITICAL-this is request 4 with [4, 161] available rides.
30-05-23 11:51:25-CRITICAL-ride 4 available [1]
30-05-23 11:51:25-CRITICAL-ride 161 available [1, 0]
30-05-23 11:51:25-CRITICAL-this is reuqest 4 with [4, 161] still available rides.
Profit Maximization
30-05-23 11:51:25-CRITICAL-vehicle 10 has 2 choices
0.5
0.5
30-05-23 11:51:25-CRITICAL-this is request 6 with [6, 122, 151, 163, 171, 215, 216, 217] available rides.
30-05-23 11:51:25-CRITICAL-ride 6 available [1]
30-05-23 11:51:25-CRITICAL-ride 122 available [1, 0]
30-05-23 11:51:25-CRITICAL-ride 151 available [1, 0]
30-05-23 11:51:25-CRITICAL-ride 163 not available [1, 3]
30-05-23 11:51:25-CRITICAL-ride 171 available [1, 0]
30-05-23 11:51:25-CRITICAL-ride 215 available [1, 0]

In [106]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM,REVENUE
veh,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,False,0,0,1794.0,45.0,0,219.0,0,...,0,0,7211.0,748.0,0,632.0,414.0,5417.0,12.332,3.0936
2,3.0,0.0,False,0,0,2576.0,45.0,0,60.0,0,...,0,0,7061.0,702.0,0,265.0,195.0,5899.0,7.040,3.0468
3,4.0,0.0,False,0,0,2544.0,45.0,0,114.0,0,...,0,0,6617.0,814.0,0,545.0,279.0,4979.0,11.641,6.8697
4,3.0,0.0,False,0,0,2323.0,30.0,0,78.0,0,...,0,0,7091.0,945.0,0,242.0,198.0,5706.0,12.049,4.4076
5,5.0,0.0,False,0,0,1669.0,45.0,0,175.0,0,...,0,0,7570.0,710.0,0,480.0,370.0,6010.0,14.113,0.9144
6,2.0,0.0,False,0,0,995.0,30.0,0,23.0,0,...,0,0,9349.0,321.0,0,21.0,113.0,8894.0,3.224,0.7788
7,3.0,0.0,False,0,0,2764.0,30.0,0,129.0,0,...,0,0,7107.0,454.0,0,225.0,249.0,6179.0,7.488,5.5404
8,3.0,0.0,False,0,0,1851.0,45.0,0,51.0,0,...,0,0,7665.0,886.0,0,211.0,186.0,6382.0,8.877,2.4456
9,2.0,0.0,False,0,0,2064.0,30.0,0,40.0,0,...,0,0,7733.0,632.0,0,240.0,130.0,6731.0,6.326,3.3660


In [107]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,LOSES_PATIENCE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,90.0,20.0,30.0,234.0,0.0,0.0,10.0,0,0,0,0,0,0,90.0,234.0,55.0,False
1,15.0,200.0,20.0,30.0,122.0,0.0,57.0,10.0,0,0,0,0,0,0,200.0,122.0,55.0,False
2,15.0,71.0,20.0,30.0,32.0,0.0,79.0,10.0,0,0,0,0,0,0,71.0,32.0,55.0,False
3,15.0,127.0,20.0,30.0,43.0,0.0,138.0,10.0,0,0,0,0,0,0,127.0,43.0,55.0,False
4,15.0,346.0,20.0,17.0,165.0,13.0,197.0,10.0,0,0,0,0,0,0,346.0,178.0,42.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.0,414.0,35.0,30.0,0.0,3010.0,204.0,10.0,0,0,0,0,0,0,414.0,3010.0,55.0,False
96,15.0,253.0,20.0,30.0,167.0,0.0,3253.0,10.0,0,0,0,0,0,0,253.0,167.0,55.0,False
97,15.0,700.0,20.0,30.0,33.0,19.0,3365.0,10.0,0,0,0,0,0,0,700.0,52.0,55.0,False


In [108]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')

KeyError: True

In [ ]:
#sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/veh{}.csv')
#sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/pax{}.csv')

In [ ]:
sim.res[0].veh_exp['REVENUE'].to_list()
print(sim.res[0].veh_exp['REVENUE'].to_list())



In [ ]:
total = sim.res[0].all_kpi # All driver revenue 
